In [ ]:
#pip install selenium

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
chromedriver = "C:/Users/anush/Downloads/chromedriver_win32/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.pmindia.gov.in/en/mann-ki-baat/")

ScrollNumber = 30
for i in range(1,ScrollNumber):
    driver.execute_script("window.scrollTo(1,50000)")
    time.sleep(5)

In [ ]:
file = open('DS.html', 'w', encoding='utf-8')
file.write(driver.page_source)
file.close()

driver.close()

In [ ]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()

In [ ]:
import urllib.request
import requests
from bs4 import BeautifulSoup

with open('DS.html', 'r', encoding='utf-8') as f:
    page = f.read()

parser = BeautifulSoup(page, 'html.parser')

links = []
for i in parser.find_all('div', attrs={'class' : 'mkb-news-wrapper'}):
    links.append(i.a['href'])

len(links)

In [ ]:
for i in range(len(links)) :
    speech_response =  requests.get(links[i])
    speech_content = speech_response.content
#     file = open('All_speech/speech'+str(i)+'.html', 'w', encoding='utf-8')
#     file.write(speech_response.content)
#     file.close()

#     with open('All_speech/speech'+str(i)+'.html', "w", encoding='utf-8') as f:
#       f.write(speech_response.content)
    
    save_html(speech_response.content, 'All_speech/speech'+str(len(links)-i)+'.html')
    file.close()
    
    time.sleep(10)
    speech_parser = BeautifulSoup(speech_content, 'html.parser')

    content = speech_parser.select(".content-block")
    #print(content)
   

In [ ]:
speech_date = []
speech = []

for i in range(len(links)) :
    html = open_html('All_speech/speech'+str(i+1)+'.html')
    parser = BeautifulSoup(html, 'html.parser')
    speech_text = parser.select('.left-container')
    for s in speech_text : 
        date = s.select(".share_date .date")[0].text
        speech_date.append(date)
        speech_ = parser.select(".news-bg")[0].text
        speech.append(speech_)

In [ ]:
import pandas as pd

col1 = speech_date
col2 = speech
mann_ki_baat = pd.DataFrame(
    {'Speech_date': col1,
     'Speech': col2
    })
mann_ki_baat

In [ ]:
cleaned = []
for date in mann_ki_baat['Speech_date']:
    clean = str(date).replace('\\n', '')
    clean = clean.replace("\'", '')
    cleaned.append(clean)
    
mann_ki_baat['Speech_date'] = cleaned

cleaned = []
for sp in mann_ki_baat['Speech']:
    clean = str(sp).replace('\n', '').replace("\'", '')
    clean = clean.replace('(Hon’ble Shri Narendra Modi):', '').replace('\\xa0', ' ')
    cleaned.append(clean)
    
mann_ki_baat['Speech'] = cleaned
mann_ki_baat

In [ ]:
mann_ki_baat.to_csv("mann_ki_baat.csv")

# LDA
---
### Topic modelling
Topic Modeling is a process to automatically identify topics present in a text object and to derive hidden patterns exhibited by a text corpus. Topic Models are very useful for multiple purposes, including:

- Document clustering
- Organizing large blocks of textual data
- Information retrieval from unstructured text
- Feature selection

Topic Modelling is different from rule-based text mining approaches that use regular expressions or dictionary based keyword searching techniques. It is an unsupervised approach used for finding and observing the bunch of words (called “topics”) in large clusters of texts.

### LDA - Latent Dirichlet Allocation
There are many approaches for obtaining topics from a text such as – Term Frequency and Inverse Document Frequency. NonNegative Matrix Factorization techniques. Latent Dirichlet Allocation is the most popular topic modeling technique.

In [1]:
import pandas as pd
import re
import string
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora

all_speeches = pd.read_csv('mann_ki_baat.csv', encoding='UTF-8')
all_speeches = all_speeches.drop(['Unnamed: 0'], axis= 1)
all_speeches

,Speech_date,Speech
0,"03 Oct, 2014","My Dear Countrymen,Today is the holy festival ..."
1,"02 Nov, 2014","My dear fellow countrymen, I am with you again..."
2,"14 Dec, 2014","My Dear Fellow Countrymen,Today I have this gr..."
3,"27 Jan, 2015","Today, Shri Barack Obama, President of the Uni..."
4,"22 Feb, 2015","Hello, my young friends. Today probably the en..."
...,...,...
68,"29 Nov, 2020","My dear countrymen,Namaskar! I want to share a..."
69,"27 Dec, 2020","My dear countrymen,Namaskar. Today is the 27th..."
70,"31 Jan, 2021","My dear countrymen,Namaskar. When I express Ma..."
71,"28 Feb, 2021","My dear countrymen,Namaskar. Yesterday was the..."


In [ ]:
# from nltk.stem import WordNetLemmatizer
# # nltk.download('wordnet')
# from nltk.tokenize import word_tokenize
# lemmatizer=WordNetLemmatizer()
# lema_words = []

# for word in result3:
#     lema_words.append(lemmatizer.lemmatize(word))

In [20]:
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])

def lemmatization(text, allowed_postags=['PROPN', 'NOUN', 'ADJ']):
    output = []
    for sent in text :
        doc = nlp(sent)
#         output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#         output.append([token for token in doc])
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return output

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
from nltk.tokenize import sent_tokenize

tokenized_sent=sent_tokenize(all_speeches['Speech'][0])
print(tokenized_sent)

['My Dear Countrymen,Today is the holy festival of Vijay Dashami.', 'My heartiest greetings on this occasion of Vijay Dashami to one and all.Through the medium of radio, I would like to share few heartfelt thoughts with you today.', 'And, I hope that not only today, this series of conversation may be carried out regularly in future.', 'I will try my best, if possible, to take out time twice a month or even once to speak with you.', 'In future, I have also decided that whenever I will speak to you, it would be on Sundays and time would be morning 11.', 'In that way, it would be convenient for you too and I will feel contented with the fact that I am successful in sharing my thoughts with you.We are celebrating the festival of Vijay Dashami today, which symbolises the triumph of Good over Evil.', 'But one gentleman named Ganesh Venkatadari, a native of Mumbai, sent me a mail and has written to me that we must take a vow to eliminate ten bad habits from within ourselves on this occasion o

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
# for i in range(len(all_speeches)) :
tokenized = lemmatization(tokenized_sent)
print(tokenized)

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['Dear', 'Countrymen', 'today', 'holy', 'festival', 'Vijay', 'Dashami'], ['heartiest', 'greeting', 'occasion', 'Vijay', 'Dashami', 'medium', 'radio', 'few', 'heartfelt', 'thought', 'today'], ['today', 'series', 'conversation', 'future'], ['good', 'possible', 'time', 'month'], ['future', 'Sundays', 'time', 'morning'], ['way', 'convenient', 'fact', 'successful', 'thought', 'festival', 'Vijay', 'Dashami', 'today', 'triumph', 'good', 'Evil'], ['gentleman', 'Ganesh', 'Venkatadari', 'native', 'Mumbai', 'mail', 'vow', 'bad', 'habit', 'occasion', 'Vijay', 'Dashami'], ['gratitude', 'recommendation'], ['individual', 'end', 'bad', 'habit', 'respect', 'nation', 'vow', 'dirt', 'country'], ['occasion', 'Vijay', 'Dashami', 'vow', 'dirt', 'filth', 'occasion', 'yesterday', '2nd', 'October', 'eve', 'Mahatma', 'Gandhi', 'birth', 'anniversary', 'more', 'crore', 'countryman', 'Swachh', 'Bharat', 'movement'], ['thought', 'yesterday', 'people', 'video', 'nation', 'social', 'medium', 'website', 'more', 'peop

In [24]:
stop_words = set(stopwords.words('english'))
result = []
for i in range(len(tokenized)) :
    tokens = tokenized[i]
    result.append([i for i in tokens if not i in stop_words])
print(result)


[['Dear', 'Countrymen', 'today', 'holy', 'festival', 'Vijay', 'Dashami'], ['heartiest', 'greeting', 'occasion', 'Vijay', 'Dashami', 'medium', 'radio', 'heartfelt', 'thought', 'today'], ['today', 'series', 'conversation', 'future'], ['good', 'possible', 'time', 'month'], ['future', 'Sundays', 'time', 'morning'], ['way', 'convenient', 'fact', 'successful', 'thought', 'festival', 'Vijay', 'Dashami', 'today', 'triumph', 'good', 'Evil'], ['gentleman', 'Ganesh', 'Venkatadari', 'native', 'Mumbai', 'mail', 'vow', 'bad', 'habit', 'occasion', 'Vijay', 'Dashami'], ['gratitude', 'recommendation'], ['individual', 'end', 'bad', 'habit', 'respect', 'nation', 'vow', 'dirt', 'country'], ['occasion', 'Vijay', 'Dashami', 'vow', 'dirt', 'filth', 'occasion', 'yesterday', '2nd', 'October', 'eve', 'Mahatma', 'Gandhi', 'birth', 'anniversary', 'crore', 'countryman', 'Swachh', 'Bharat', 'movement'], ['thought', 'yesterday', 'people', 'video', 'nation', 'social', 'medium', 'website', 'people'], ['nation', 'peopl

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
dictionary = corpora.Dictionary(tokenized)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized]

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
LDA = gensim.models.ldamodel.LdaModel

lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, 
                random_state=100, chunksize=1, passes=50, iterations=10)

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
lda_model.print_topics()

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.145*"strength" + 0.035*"home" + 0.031*"skill" + 0.030*"medium" + 0.021*"development" + 0.021*"course" + 0.020*"inner" + 0.018*"hut" + 0.018*"village" + 0.018*"Youths"'),
 (1,
  '0.153*"suggestion" + 0.044*"countryman" + 0.037*"people" + 0.033*"crore" + 0.017*"numerous" + 0.017*"advice" + 0.017*"am" + 0.017*"love" + 0.016*"Sundays" + 0.013*"effective"'),
 (2,
  '0.091*"speech" + 0.091*"original" + 0.050*"step" + 0.047*"version" + 0.047*"authoritative" + 0.044*"english" + 0.044*"rendering" + 0.044*"Hindi" + 0.028*"corner" + 0.014*"Dashami"'),
 (3,
  '0.296*"nation" + 0.039*"trust" + 0.022*"Sisters" + 0.022*"poor" + 0.017*"height" + 0.017*"new" + 0.016*"farmer" + 0.016*"village" + 0.016*"Youths" + 0.016*"hut"'),
 (4,
  '0.125*"thought" + 0.049*"today" + 0.047*"time" + 0.035*"heartfelt" + 0.030*"journey" + 0.027*"heart" + 0.023*"fortunate" + 0.023*"beginning" + 0.019*"incident" + 0.018*"future"')]

In [54]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
vis

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.162962 -0.060841       1        1  26.410604
1     -0.111871 -0.008952       2        1  26.011924
2      0.032558  0.215845       3        1  17.688874
3      0.144174 -0.078480       4        1  16.164470
0      0.098100 -0.067573       5        1  13.724128, topic_info=           Term       Freq      Total Category  logprob  loglift
45       nation  26.000000  26.000000  Default  30.0000  30.0000
187  suggestion  21.000000  21.000000  Default  29.0000  29.0000
14      thought  18.000000  18.000000  Default  28.0000  28.0000
110    strength  11.000000  11.000000  Default  27.0000  27.0000
288      speech   9.000000   9.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
120     Sisters   0.903267   3.293436   Topic5  -4.3942   0.6923
275       Radio   0.751389   3.087306   Topic5  -4.5783   0.5729
276        glad   0.751389   3.087306   Topic5  -4.5783   0.5729
16       future   1.128695   5.022377   Topic5  -4.1714   0.4932
277      simple   0.751389   3.087306   Topic5  -4.5783   0.5729

[206 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
235       5  0.740844         5th
1         3  0.423641     Dashami
225       4  0.523789  Government
185       2  0.670518     Gujarat
284       3  0.840687       Hindi
...     ...       ...         ...
244       5  0.711340     various
290       3  0.782219     version
283       4  0.307556     village
283       5  0.307556     village
29        1  0.582350         way

[151 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 4, 1])

# LDA for all Speeches 
---
Get an overall idea of the topics discussed in Mann Ki Baat

In [2]:
from nltk.tokenize import sent_tokenize

# for i in range(len(all_speeches)) :
all_speeches['Tokenized_sent'] = all_speeches['Speech'].apply(sent_tokenize)

In [3]:
all_speeches

,Speech_date,Speech,Tokenized_sent
0,"03 Oct, 2014","My Dear Countrymen,Today is the holy festival ...","[My Dear Countrymen,Today is the holy festival..."
1,"02 Nov, 2014","My dear fellow countrymen, I am with you again...","[My dear fellow countrymen, I am with you agai..."
2,"14 Dec, 2014","My Dear Fellow Countrymen,Today I have this gr...","[My Dear Fellow Countrymen,Today I have this g..."
3,"27 Jan, 2015","Today, Shri Barack Obama, President of the Uni...","[Today, Shri Barack Obama, President of the Un..."
4,"22 Feb, 2015","Hello, my young friends. Today probably the en...","[Hello, my young friends., Today probably the ..."
...,...,...,...
68,"29 Nov, 2020","My dear countrymen,Namaskar! I want to share a...","[My dear countrymen,Namaskar!, I want to share..."
69,"27 Dec, 2020","My dear countrymen,Namaskar. Today is the 27th...","[My dear countrymen,Namaskar., Today is the 27..."
70,"31 Jan, 2021","My dear countrymen,Namaskar. When I express Ma...","[My dear countrymen,Namaskar., When I express ..."
71,"28 Feb, 2021","My dear countrymen,Namaskar. Yesterday was the...","[My dear countrymen,Namaskar., Yesterday was t..."


In [4]:
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])

import itertools

def lemmatization(text, allowed_postags=['PROPN', 'NOUN', 'ADJ']):
    output = []
    for sent in text :
        doc = nlp(sent)
#         output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#         output.append([token for token in doc])
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    
    final = list(itertools.chain(*output))
    return final

In [5]:
# for i in range(len(all_speeches)) :
stop_words = set(stopwords.words('english'))
final_tokenzied = []
for i in range(len(all_speeches)) :
    tokens = lemmatization(all_speeches['Tokenized_sent'][i])
    final_tokenzied.append([i for i in tokens if not i in stop_words])

In [6]:
# final = list(itertools.chain(*final_tokenzied))
# final

In [7]:
dictionary_all = corpora.Dictionary(final_tokenzied)
doc_term_matrix_all = [dictionary_all.doc2bow(rev) for rev in final_tokenzied]

In [9]:
LDA = gensim.models.ldamodel.LdaModel

lda_model_all = LDA(corpus=doc_term_matrix_all, id2word=dictionary_all, num_topics=5, 
                random_state=100, chunksize=2, passes=50, iterations=100)
lda_model_all.save('lda.model')

In [10]:
lda_model_all.print_topics()

[(0,
  '0.011*"Ravidas" + 0.008*"commentary" + 0.008*"cook" + 0.008*"brinjal" + 0.006*"king" + 0.006*"storytelling" + 0.005*"Ismail" + 0.005*"Nayak" + 0.005*"Magh" + 0.005*"specie"'),
 (1,
  '0.021*"country" + 0.020*"India" + 0.017*"people" + 0.015*"many" + 0.014*"time" + 0.012*"countryman" + 0.012*"year" + 0.012*"new" + 0.010*"day" + 0.010*"life"'),
 (2,
  '0.024*"farmer" + 0.020*"water" + 0.017*"Corona" + 0.015*"friend" + 0.013*"light" + 0.008*"Sir" + 0.008*"sir" + 0.007*"village" + 0.007*"sister" + 0.006*"period"'),
 (3,
  '0.012*"Guru" + 0.008*"Nanak" + 0.008*"khadi" + 0.007*"Sardar" + 0.007*"Dev" + 0.007*"dog" + 0.006*"ji-" + 0.006*"Patel" + 0.005*"Unity" + 0.005*"cigarette"'),
 (4,
  '0.017*"woman" + 0.011*"yoga" + 0.011*"spirit" + 0.010*"nature" + 0.009*"Modi" + 0.008*"book" + 0.008*"duty" + 0.008*"crisis" + 0.007*"scheme" + 0.006*"-"')]

In [39]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_all, doc_term_matrix_all, dictionary_all)
vis

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.366805  0.028707       1        1  67.070808
2     -0.088066 -0.259141       2        1  14.610304
4     -0.155951  0.202571       3        1  13.547862
3     -0.065876  0.019459       4        1   3.054601
0     -0.056912  0.008405       5        1   1.716424, topic_info=          Term         Freq        Total Category  logprob  loglift
119     farmer   336.000000   336.000000  Default  30.0000  30.0000
275      water   281.000000   281.000000  Default  29.0000  29.0000
124     friend   670.000000   670.000000  Default  28.0000  28.0000
82     country  1352.000000  1352.000000  Default  27.0000  27.0000
20       India  1261.000000  1261.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
10453   potato     3.997102     4.857917   Topic5  -6.0172   3.8699
8926    specie     8.724535    10.764575   Topic5  -5.2367   3.8548
10978     Chia     4.844650     5.982604   Topic5  -5.8249   3.8539
3748      king     9.593556    35.351100   Topic5  -5.1417   2.7607
10362     Rama     6.637737    15.963771   Topic5  -5.5100   3.1874

[208 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
559       3  0.993466      -
2069      1  0.380993   31st
2069      4  0.634989   31st
9356      5  0.907374   Amma
563       3  0.988008  Assam
...     ...       ...    ...
871       1  0.995223   work
280       1  0.998623  world
553       1  0.999200   year
875       3  0.999709   yoga
554       1  0.999213  young

[196 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 5, 4, 1])

# Sentiment Analysis
---

In [ ]:
def preprocess(text):
    input_str = text.lower()
    input_str = re.sub(r'\d+', '', input_str)
    input_str = input_str.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    result = input_str.strip()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(result)
    result = [i for i in tokens if not i in stop_words]
    
    remove = ['’','‘','“', '”', 'th', 'nd', 'st','ji','ii']
    final = []
    for word in result :
        if word not in remove :
            final.append(word)
        else :
            pass
    return result

In [ ]:
all_speeches['Preprocessed'] = all_speeches['Speech'].apply(preprocess)

In [ ]:
all_speeches


In [40]:
import text2emotion as te

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
text = all_speeches['Speech'][0]

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
te.get_emotion(all_speeches['Speech'][0])

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'Happy': 0.13, 'Angry': 0.04, 'Surprise': 0.26, 'Sad': 0.22, 'Fear': 0.35}

In [53]:
print(all_speeches['Speech_date'][60:])

60    29 Mar, 2020
61    26 Apr, 2020
62    31 May, 2020
63    28 Jun, 2020
64    26 Jul, 2020
65    30 Aug, 2020
66    27 Sep, 2020
67    25 Oct, 2020
68    29 Nov, 2020
69    27 Dec, 2020
70    31 Jan, 2021
71    28 Feb, 2021
72    28 Mar, 2021
Name: Speech_date, dtype: object


C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
